# Geometry overview

Earth Engine handles vector data with the `Geometry` type. The [GeoJSON spec](https://geojson.org/geojson-spec.html) describes in detail the type of geometries supported by Earth Engine, including `Point` (a list of coordinates in some projection), `LineString` (a list of points), `LinearRing` (a closed `LineString`), and `Polygon` (a list of `LinearRings` where the first is a shell and subsequent rings are holes). Earth Engine also supports `MultiPoint`, `MultiLineString`, and `MultiPolygon`. The GeoJSON GeometryCollection is also supported, although it has the name `MultiGeometry` within Earth Engine.


#### Creating Geometry objects
 To create a Geometry programmatically, provide the constructor with the proper list(s) of coordinates. For example:

In [ ]:
point = ee.Geometry.Point([1.5, 1.5])
lineString = ee.Geometry.LineString([[-35, -10], [35, -10], [35, 10], [-35, 10]])
linearRing = ee.Geometry.LinearRing([[-35, -10], [35, -10], [35, 10], [-35, 10], [-35, -10]])
rectangle = ee.Geometry.Rectangle([-40, -20, 40, 20])
polygon = ee.Geometry.Polygon([[[-5, 40], [65, 40], [65, 60], [-5, 60], [-5, 60]]])

In the previous examples, note that the distinction between a `LineString` and a `LinearRing` is that the `LinearRing` is “closed” by having the same coordinate at both the start and end of the list.

An individual `Geometry` may consist of multiple geometries. To break a multi-part `Geometry` into its constituent geometries, use `geometry.geometries()`. For example:

In [ ]:
# Create a multi-part feature.
multiPoint = ee.Geometry.MultiPoint([[-121.68, 39.91], [-97.38, 40.34]])

# Get the individual geometries as a list.
geometries = multiPoint.geometries()

# Get each individual geometry from the list and print it.
pt1 = geometries.get(0)
pt2 = geometries.get(1)
print('Point 1', pt1)
print('Point 2', pt2)

# Geodesic vs. Planar Geometries

A geometry created in Earth Engine is either geodesic (i.e. edges are the shortest path on the surface of a sphere) or planar (i.e. edges are the shortest path in a 2-D Cartesian plane). No one planar coordinate system is suitable for global collections of features, so Earth Engine's geometry constructors build geodesic geometries by default. To make a planar geometry, constructors have a geodesic parameter that can be set to false:



In [ ]:
planarPolygon = ee.Geometry(polygon, None, False)

Figure 1 shows the difference between the default geodesic polygon and the result of converting the polygon to a planar representation.


<center>
<img src="https://developers.google.com/earth-engine/images/Geometry_geodesic_vs_planar_annotated.png">  
</center>

# Geometry information and metadata

To view information about a geometry, print it. To access the information programmatically, Earth Engine provides several methods. For example, to get information about the polygon created previously, use:

In [ ]:
print('Polygon printout: ', polygon.getInfo())

# Print polygon area in square kilometers.
print('Polygon area: ', polygon.area().divide(1000 * 1000).getInfo())

# Print polygon perimeter length in kilometers.
print('Polygon perimeter: ', polygon.perimeter().divide(1000).getInfo())

# Print the geometry as a GeoJSON string.
print('Polygon GeoJSON: ', polygon.toGeoJSONString())

# Print the GeoJSON 'type'
print('Geometry type: ', polygon.type().getInfo())

# Print the coordinates as lists
print('Polygon coordinates: ', polygon.coordinates().getInfo())

# Print whether the geometry is geodesic.
print('Geodesic? ', polygon.geodesic().getInfo())      

Observe that the perimeter (or length) of a geometry is returned in meters and the area is returned in square meters unless a projection is specified. By default, the computation is performed on the WGS84 spheroid and the result is computed in meters or square meters.

# Geometric Operations

Earth Engine supports a wide variety of operations on `Geometry` objects. These include operations on individual geometries such as computing a buffer, centroid, bounding box, perimeter, convex hull, etc. For example:



In [ ]:
# Create a geodesic polygon.
polygon = ee.Geometry.Polygon(coords = [[[-5, 40], [65, 40], [65, 60], [-5, 60], [-5, 60]]])

# Compute a buffer of the polygon.
buffer = polygon.buffer(1000000).getInfo()

# Compute the centroid of the polygon.
centroid = polygon.centroid().getInfo()


#Go Folium
center = centroid['coordinates']

Mapdisplay(center,{'center':centroid,'buffer':buffer},zoom_start=3)

Observe from the previous example that the buffer distance is specified in meters.

Supported geometric operations also include relational computations between geometries such as intersection, union, difference, distance, contains, etc. To test some of these relations, geometries use the “even-odd” rule by default. By the even-odd rule, a point is inside the polygon if a line from that point to some point known to be outside the polygon crosses an odd number of other edges. The inside of a polygon is everything inside the shell and not inside a hole. As a simple example, a point within a circular polygon must cross exactly one edge to escape the polygon. Geometries can optionally use the "left-inside" rule, if necessary. Imagine walking the points of a ring in the order given; the inside will be on the left.

To demonstrate the difference between geometries created with the "left-inside" rule (`evenOdd: false`) and those created with the "even-odd" rule, the following example compares a point to two different polygons:

In [ ]:
# Create a right-inside polygon.
holePoly = ee.Geometry.Polygon(coords = [[[-35, -10], [-35, 10], [35, 10], [35, -10], [-35, -10]]],
                               proj= 'EPSG:4326',
                               geodesic = True,
                               maxError= 1.,
                               evenOdd = False)

# Create an even-odd version of the polygon.
evenOddPoly = ee.Geometry(geo_json =  holePoly.getInfo(),
                          opt_proj = 'EPSG:4326',
                          opt_evenOdd = True)

# Create a point to test the insideness of the polygon.
pt = ee.Geometry.Point([1.5, 1.5])

# Check insideness with a contains operator.
print(holePoly.contains(pt).getInfo())       # False
print(evenOddPoly.contains(pt).getInfo())    # True

The previous example demonstrates how the order of coordinates provided to the Polygon constructor affects the result when a left-inside polygon is constructed. Specifically, the point is outside the left-inside polygon but inside the even-odd polygon.

The following example computes and visualizes derived geometries based on the relationship between two polygons:

In [ ]:
# Create two circular geometries.
poly1 = ee.Geometry.Point([-50, 30]).buffer(1e6)
poly2 = ee.Geometry.Point([-40, 30]).buffer(1e6)

poly1_geojson = poly1.getInfo()
poly2_geojson = poly2.getInfo()

# Display polygon 1 in red and polygon 2 in blue.
Mapdisplay(center= (30,-45),dicc={'poly1':poly1_geojson,'poly2':poly2_geojson},zoom_start=3)

In [ ]:
# Compute the intersection, display it in blue.
intersection = poly1.intersection(poly2, ee.ErrorMargin(1)).getInfo()
Mapdisplay(center= (30,-45),dicc={'intersection':intersection},zoom_start=3)

In [ ]:
# Compute the union, display it in magenta.
union = poly1.union(poly2, ee.ErrorMargin(1)).getInfo()
Mapdisplay(center= (30,-45),dicc={'union':union},zoom_start=3)

In [ ]:
# Compute the difference, display in yellow.
diff1 = poly1.difference(poly2, ee.ErrorMargin(1)).getInfo()
Mapdisplay(center= (30,-45),dicc={'diff':diff1},zoom_start=3)

In [ ]:
# Compute symmetric difference, display in black.
symDiff = poly1.symmetricDifference(poly2, ee.ErrorMargin(1)).getInfo()
Mapdisplay(center= (30,-45),dicc={'symDiff':symDiff},zoom_start=3)

In these examples, note that that maxError parameter is set to one meter for the geometry operations. The maxError is the maximum allowable error, in meters, from transformations (such as projection or reprojection) that may alter the geometry. If one of the geometries is in a different projection from the other, Earth Engine will do the computation in a spherical coordinate system, with a projection precision given by maxError. You can also specify a specific projection in which to do the computation, if necessary.



# Feature Overview

A `Feature` in Earth Engine is defined as a GeoJSON `Feature`. Specifically, a `Feature` is an object with a geometry property storing a `Geometry` object (or null) and a `properties` property storing a dictionary of other properties.

## Creating Feature objects
To create a `Feature`, provide the constructor with a `Geometry` and (optionally) a dictionary of other properties. For example:

In [ ]:
# Create an ee.Geometry.
polygon = ee.Geometry.Polygon([[[-35, -10], [35, -10], [35, 10], [-35, 10], [-35, -10]]])

# Create a Feature from the Geometry.
polyFeature = ee.Feature(polygon, {'foo': 42, 'bar': 'tart'})

As with a `Geometry`, a `Feature` may be printed or added to the map for inspection and visualization:

In [ ]:
polyFeature.getInfo()

A `Feature` need not have a Geometry and may simply wrap a dictionary of properties. For example:



In [ ]:
# Create a dictionary of properties, some of which may be computed values.
dict = {'foo': ee.Number(8).add(88), 'bar': 'nihao'}

# Create a null geometry feature with the dictionary of properties.
nowhereFeature = ee.Feature(None, dict)

In this example, note that the dictionary supplied to the `Feature` contains a computed value. Creating features in this manner is useful for exporting long-running computations with a `Dictionary` result (e.g. `image.reduceRegion()`). See the [FeatureCollections](https://developers.google.com/earth-engine/feature_collections) and [Importing](https://developers.google.com/earth-engine/importing) or [Exporting](https://developers.google.com/earth-engine/exporting) sections for details.

Each `Feature` has one primary `Geometry` stored in the `geometry` property. Additional geometries may be stored in other properties. `Geometry` methods such as intersection and buffer also exist on `Feature` as a convenience for getting the primary `Geometry`, applying the operation, and setting the result as the new primary `Geometry`. The result will retain all the other properties of the `Feature` on which the method is called. There are also methods for getting and setting the non-geometry properties of the `Feature`. For example:

In [ ]:
from pprint import pprint
# Make a feature and set some properties.
feature = ee.Feature(ee.Geometry.Point([-122.22599, 37.17605]))\
            .set('genus', 'Sequoia').set('species', 'sempervirens')

# Get a property from the feature.
species = feature.get('species')
print(species.getInfo())

# Set a new property.
feature = feature.set('presence', 1)

# Overwrite the old properties with a new dictionary.
newDict = {'genus': 'Brachyramphus', 'species': 'marmoratus'}
feature = feature.set(newDict)

# Check the result.
pprint(feature.getInfo())

In the previous example, note that properties can be set with either a key: value pair, or with a dictionary as a JavaScript literal. Also note that `feature.set()` overwrites existing properties.

## Creating your own GeoJSON
Basically a simple syntax to store geographical data. For more information, take a look at [this video](https://youtu.be/8RPfrhzRw2s). When using GEE or other GIS software it can often be helpful to create your own GeoJSONs quickly. You can use this to quickly create a polygon for a area you are interested in. To do this we can use [geojson.io](http://geojson.io/#map=2/20.0/0.0) ([explanation video](https://youtu.be/9ZkXYnl3-yg)). If you want to use it in Google Colab, we need to copy the output of the website. 

In [ ]:
my_geojson = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -12.83203125,
              13.325484885597936
            ],
            [
              16.5234375,
              13.325484885597936
            ],
            [
              16.5234375,
              33.211116472416855
            ],
            [
              -12.83203125,
              33.211116472416855
            ],
            [
              -12.83203125,
              13.325484885597936
            ]
          ]
        ]
      }
    }
  ]
}

To convert it to a usable geometry, we can just use geemap. 

In [ ]:
area_of_interest = geemap.geojson_to_ee(my_geojson)
print(area_of_interest)

# FeatureCollection Overview

Groups of related features can be combined into a `FeatureCollection`, to enable additional operations on the entire set such as filtering, sorting and rendering. Besides just simple features (geometry + properties), feature collections can also contain other collections.

#### The FeatureCollection constructor

One way to create a `FeatureCollection` is to provide the constructor with a list of features. The features do not need to have the same geometry type or the same properties. For example:

In [ ]:
# Make a list of Features.
features = [
  ee.Feature(ee.Geometry.Rectangle(30.01, 59.80, 30.59, 60.15), {'name': 'Voronoi'}),
  ee.Feature(ee.Geometry.Point(-73.96, 40.781), {'name': 'Thiessen'}),
  ee.Feature(ee.Geometry.Point(6.4806, 50.8012), {'name': 'Dirichlet'})
]

# Create a FeatureCollection from the list and print it.
fromList = ee.FeatureCollection(features)
fromList.getInfo()

Individual geometries can also be turned into a `FeatureCollection` of just one Feature:



In [ ]:
# Create a FeatureCollection from a single geometry and print it.
fromGeom = ee.FeatureCollection(ee.Geometry.Point(16.37, 48.225))
fromGeom.getInfo()

## Table Datasets

Earth Engine hosts a variety of table datasets. To load a table dataset, provide the table ID to the `FeatureCollection` constructor. For example, to load TIGER roads data:

In [ ]:
fc = ee.FeatureCollection('TIGER/2016/Roads')
road = fc.limit(50)
centroid = road.geometry().centroid().getInfo()['coordinates']
centroid.reverse()
Mapdisplay(centroid,{'road':road.getInfo()},zoom_start=12)

## Random Samples

To get a collection of random points in a specified region, you can use:

In [ ]:
# Define an arbitrary region in which to compute random points.
region = ee.Geometry.Rectangle(-119.224, 34.669, -99.536, 50.064)
center_coord = region.centroid().getInfo()['coordinates']
center_coord.reverse()

# Create 1000 random points in the region.
randomPoints = ee.FeatureCollection.randomPoints(region=region,points=100)

# Display the points.
Mapdisplay(center_coord,{'randomPoints':randomPoints.getInfo()},zoom_start=5)

# FeatureCollection Information and Metadata

Methods for getting information from image collections are directly applicable to feature collections. (See the [ImageCollection Information and Metadata section](https://developers.google.com/earth-engine/ic_info) for details). You can use the aggregation shortcuts to count the number of features or summarize an attribute:



In [ ]:
from pprint import pprint
# Load watersheds from a Fusion Table.
sheds = ee.FeatureCollection('ft:1IXfrLpTHX4dtdj1LcNXjJADBB-d93rkdJ9acSEWK')

#Select just 10 features
sheds10 = sheds.limit(10)
sheds_centroid = sheds10.geometry().centroid().getInfo()['coordinates']
sheds_centroid.reverse()

# Display the table and print its first element.
pprint(sheds10.limit(1).getInfo()['features'][0]['properties'])

# Print the number of watersheds.
print('\n')
print('Count: ', sheds10.size().getInfo())

# Print stats for an area property.
print('\n')
pprint(sheds10.aggregate_stats('AreaSqKm').getInfo())

In [ ]:
Mapdisplay(sheds_centroid,{'sheds':sheds10.getInfo()},zoom_start=5)

For more general purpose FeatureCollection aggregation tools, see the Reducing a FeatureCollection section.



#8. Filtering a FeatureCollection

Filtering a `FeatureCollection` is analogous to filtering an `ImageCollection`. (See the Filtering an ImageCollection section). There are the `featureCollection.filterDate()`, and `featureCollection.filterBounds()` convenience methods and the `featureCollection.filter()` method for use with any applicable `ee.Filter`. For example:


In [ ]:
# Load watersheds from a Fusion Table.
sheds = ee.FeatureCollection('ft:1IXfrLpTHX4dtdj1LcNXjJADBB-d93rkdJ9acSEWK')

# Define a region roughly covering the continental US.
continentalUS = ee.Geometry.Rectangle(-127.18, 19.39, -62.75, 51.29)

# Filter the table geographically: only watersheds in the continental US.
filtered = sheds.filterBounds(continentalUS)

# Check the number of watersheds after filtering for location.
print('Count after filter:', filtered.size().getInfo())

# Filter to get only larger continental US watersheds.
largeSheds = filtered.filter(ee.Filter.gt('AreaSqKm', 25000))

# Check the number of watersheds after filtering for size and location.
print('Count after filtering by size:', largeSheds.size().getInfo())

# Mapping over a FeatureCollection

To apply the same operation to every `Feature` in a `FeatureCollection`, use `featureCollection.map()`. For example, to add another area attribute to every feature in a watersheds `FeatureCollection`, use:



In [ ]:
# Load watersheds from a Fusion Table.
sheds = ee.FeatureCollection('ft:1IXfrLpTHX4dtdj1LcNXjJADBB-d93rkdJ9acSEWK')

# This function computes the feature's geometry area and adds it as a property.
def addArea(feature):
  return feature.set({'areaHa': feature.geometry().area().divide(100 * 100)})

# Map the area getting function over the FeatureCollection.
areaAdded = sheds.map(addArea)

# Print the first feature from the collection with the added property.
print('First feature: \n')
pprint(areaAdded.first().getInfo())

In the previous example, note that a new property is set based on a computation with the feature’s geometry. Properties can also be set using a computation involving existing properties.

An entirely new `FeatureCollection` can be generated with `map()`. The following example converts the watersheds to centroids:

In [ ]:
# This function creates a new feature from the centroid of the geometry.
def getCentroid(feature):
  # Keep this list of properties.
  keepProperties = ['name', 'HUC6', 'TNMID', 'AreaSqKm']
  # Get the centroid of the feature's geometry.
  centroid = feature.geometry().centroid()
  # Return a new Feature, copying properties from the old Feature.
  return ee.Feature(centroid).copyProperties(feature, keepProperties)


# Map the centroid getting function over the features.
centroids = sheds.map(getCentroid)
centroids_cn = centroids.geometry().centroid().getInfo()['coordinates']
centroids_cn.reverse()

# Display the results and print the first few.
Mapdisplay(centroids_cn,{'centroids':centroids.getInfo()},zoom_start=4)

In [ ]:
print('Centroids: \n')
pprint(centroids.limit(5).getInfo())

Note that only a subset of properties is propagated to the features in the new collection.



# Reducing a FeatureCollection

To aggregate data in the properties of a `FeatureCollection`, use `featureCollection.reduceColumns()`. For example, to check the area properties in the watersheds `FeatureCollection`, this code computes the Root Mean Square Error (RMSE) relative to the Earth Engine computed area:



In [ ]:
# Load watersheds from a Fusion Table and filter to the continental US.
sheds = ee.FeatureCollection('ft:1IXfrLpTHX4dtdj1LcNXjJADBB-d93rkdJ9acSEWK')\
          .filterBounds(ee.Geometry.Rectangle(-127.18, 19.39, -62.75, 51.29))

# This function computes the squared difference between an area property
# and area computed directly from the feature's geometry.
def areaDiff(feature):
  # Compute area in sq. km. directly from the geometry.
  area = feature.geometry().area().divide(1000 * 1000)
  # Compute the differece between computed area and the area property.
  diff = area.subtract(feature.get('AreaSqKm'))
  # Return the feature with the squared difference set to the 'diff' property.
  return feature.set('diff', diff.pow(2))


# Map the difference function over the collection.
rmse = ee.Number(sheds.map(areaDiff)\
                      .reduceColumns(ee.Reducer.mean(), ['diff'])\
                      .get('mean'))\
         .sqrt()
# Print the result.
print('RMSE=', rmse.getInfo())

In this example, note that the return value of reduceColumns() is a dictionary with key ‘mean’. To get the mean, cast the result of dictionary.get() to a number with ee.Number() before trying to call sqrt() on it. For more information about ancillary data structures in Earth Engine, see the [introduction of eeCourse]().

To overlay features on imagery, use featureCollection.reduceRegions(). For example, to compute the volume of precipitation in continental US watersheds, use reduceRegions() followed by a map():

In [ ]:
# Load an image of daily precipitation in mm/day.
precip = ee.Image(ee.ImageCollection('NASA/ORNL/DAYMET_V3').first())

# Load watersheds from a Fusion Table and filter to the continental US.
sheds = ee.FeatureCollection('ft:1IXfrLpTHX4dtdj1LcNXjJADBB-d93rkdJ9acSEWK')\
          .filterBounds(ee.Geometry.Rectangle(-127.18, 19.39, -62.75, 51.29))

# Add the mean of each image as new properties of each feature.
withPrecip = precip.reduceRegions(sheds, ee.Reducer.mean())

# This function computes total rainfall in cubic meters.
def prcpVolume(feature):
  # Precipitation in mm/day -> meters -> sq. meters.
  volume = ee.Number(feature.get('prcp'))\
             .divide(1000)\
             .multiply(feature.geometry().area())
  return feature.set('volume', volume)

# 1. Map the function over the collection.
# 2. Sort descending.
# 3. Get only the 5 highest volume watersheds.
highVolume = withPrecip.map(prcpVolume)\
                       .sort('volume', False)\
                       .limit(5)

# Print the resulting FeatureCollection
pprint(highVolume.getInfo())  

For more information about reducing feature collections, see[ Statistics of FeatureCollection Columns]() and [Vector to Raster Conversion]().

## Clipping a raster by a vector in Google Earth Engine
Clipping rasters to the extents you need is one of the most common operations in GIS, which means you can also do it in GEE. For this we will use the GeoJSON file we just read in and a [DEM](https://developers.google.com/earth-engine/datasets/catalog/CGIAR_SRTM90_V4). 

In [ ]:
dem = ee.Image("CGIAR/SRTM90_V4") 
dem_clipped = dem.clip(area_of_interest)
Map = geemap.Map()
Map.addLayer(dem_clipped,{},"Area of Interest")
Map.center_object(dem_clipped)
Map

## Exercise 1: Getting feature collections 
When we use GEE we need to import the data that Google is already providing. To find the right dataset we use the [GEE Data Catalog](https://developers.google.com/earth-engine/datasets/catalog). For example, let's get the [global adminstrative units](https://developers.google.com/earth-engine/datasets/catalog/FAO_GAUL_2015_level0), so we can display the borders of states. The good thing is that this page already contains the right code snippet for  us to import this feature collection: 

In [ ]:
ee.FeatureCollection("FAO/GAUL/2015/level0")

Select [Table Schema](https://developers.google.com/earth-engine/datasets/catalog/FAO_GAUL_2015_level0#table-schema) on this site to see the different attributes of this dataset and how to access them. So, for example we can get all the country names from this by simply writing:

In [ ]:
# We get the feature collection of Administrative boundaries (level0)
countries = ee.FeatureCollection('FAO/GAUL/2015/level0').select('ADM0_NAME')

And we can also filter it to get a subset. 

In [ ]:
# We filter the featureCollection to get the feature we want
# the .eq method stands for equal. So this looks for the name Germany in the attribute ADM0_Name
germany = countries.filter(ee.Filter.eq('ADM0_NAME', 'Germany'))

And finally we can use geemap to show us the country we selected as a highlight on the map. 

In [ ]:
# Create a map
Map = geemap.Map(zoom=3)
# Add the Germany feature 
Map.addLayer(germany,{},'Countries')
# Show it
Map

Repeat this exercise with the state of the country you were born in (hint: you have to select another feature collection.

## Exercise 2: Getting info from a feature collection
As we saw in the exercise before, feature collections contain information about their polygons like their name. We used this to select states by name. We can also ask GEE to give us all the information in a feature collection, so we can use it to select the wnats we are interest in. For this we can use the .getInfo() method. 


In [ ]:
# We get the feature collection of Administrative boundaries (level0) and ask for all values in the ADM0_Name attribute
countries = ee.FeatureCollection('FAO/GAUL/2015/level0').aggregate_array("ADM0_NAME").getInfo()

Repeat this for all the states in your home country. 

## Exercise 3: Filter based on a numerical attribute
For this we will use the dataset of the [RESOLVE ecoregions](https://developers.google.com/earth-engine/datasets/catalog/RESOLVE_ECOREGIONS_2017#table-schema). This is a global mapping of ecosystem regions. We can select it with:

In [ ]:
# Get the ecoregion data 
ecoregions = ee.FeatureCollection("RESOLVE/ECOREGIONS/2017")

Now only select the mangroves worldwide and display them:

In [ ]:
# Only select mangroves
mangroves = ecoregions.filter(ee.Filter.eq('BIOME_NAME', 'Mangroves'))
# Show them 
Map = geemap.Map()
Map.addLayer(mangroves,{},"Mangroves Worldwide")
Map

Next we only want to select all mangroves that are larger than 3 square degrees:

In [ ]:
# Only select those mangroves with a total area greater than (gt) than 3 square degrees
largest_mangroves = mangroves.filter(ee.Filter.gt("SHAPE_AREA", 3.3))
# Show them 
Map = geemap.Map()
Map.addLayer(largest_mangroves,{},"Largest Mangroves Worldwide")
Map.center_object(largest_mangroves)
Map

Repeat this exercise and find the smallest "Temperate Conifer Forests" in the world. Where is it located?

## Exercise 4: Grouping feature collections by attribute
Grouping is a powerful technique. It basically allows you to sort your data by a specific attribute. For example, you could group your data by country and subsequently calculate all your other attributes on a by country basis. This allows you to get a quick overview over your data. 

To do this properly we first want to exclude all shapes with missing values, as they skew our selection. 

In [ ]:
# Get the ecoregion data 
ecoregions = ee.FeatureCollection("RESOLVE/ECOREGIONS/2017")
# Remove non values for numerical attributes 
# the .And allows to select several attributes at once
# .neq stands for not equal
data_without_none = ecoregions.filter(ee.Filter.And(ee.Filter.neq("SHAPE_AREA", None),
                                                   ee.Filter.neq("SHAPE_LENGTH",None)))

Next we group by the ecoregion and calculate the mean of the shape area, to see which ecoregion is the largest one on earth. This looks complicated, but the idea is actually pretty simple. 

In [ ]:
grouped_regions = data_without_none.reduceColumns(**{
    # Define the columns we want to work with
    "selectors": ["SHAPE_AREA", "BIOME_NAME"],
    # Define what we want to calculate, e.g. min, max, mean
    # .repeat tells GEE how many of the attributes should be used for calculation (starting with the first one)
    # .group defines that we want to group the results by something
    "reducer": ee.Reducer.mean().repeat(1).group(**{
        # the group field refers to the line with "selectors" and starts counting at 0
        # the field we select will be used for grouping, so "BIOME_NAME" in this case
        "groupField":1,
        # groupName is simply a lable we want to use for the groups
        "groupName":"Ecoregion"
    })
})
grouped_regions.getInfo()

Repeat this exercise by selecting a new feature collection from the [data calalog](https://developers.google.com/earth-engine/datasets/catalog) and collect the max value for a meaningful grouped attribute.